In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

genotype_path = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/cteph_rare_variant/pro_assoc/02.snp_to_soma/low_missing_genotype.csv"
age_sex_path = "/LARGE0/gr10478/b37974/Pulmonary_Hypertension/Proteome/case_control_prepare/sample.csv"

genotype_df = pd.read_csv(genotype_path, index_col=0)
age_sex_df = pd.read_csv(age_sex_path, index_col=0)

In [9]:
age_sex_df = age_sex_df.drop('SampleCode', axis=1)
age_sex_df['sex_male1_female2'] = age_sex_df['sex_male1_female2'].replace(2, 0)
age_sex_df = age_sex_df.rename(columns={'sex_male1_female2': 'sex'})
age_sex_df.index = age_sex_df.index.str.replace('_day0', '')
age_sex_df = age_sex_df.loc[age_sex_df.index.intersection(genotype_df.columns)]

scaler = StandardScaler()
age_sex_df['age'] = scaler.fit_transform(age_sex_df[['age']])

age_sex_df['group'] = 0
age_sex_df['group'] = age_sex_df.index.to_series().apply(lambda x: 1 if x.startswith('PHOM') else 0)

In [10]:
selected_snp = ['chr5:96783162:C:T', 'chr6:31506394:C:T', 'chr7:12570952:C:T', 
                'chr11:126338337:G:C', 'chr15:73260253:T:TCCACTTTTCCTTC', 'chr15:81319380:A:G', 
                'chr16:56982238:G:A', 'chr17:66223742:CA:C', 'chr20:23444046:T:C']

genotype_df = genotype_df.loc[selected_snp]
genotype_df = genotype_df.replace({'0/0': 0, '1/0': 1, '0/1': 1, '1/1': 2})
genotype_df = genotype_df.astype('float64')

genotype_df = genotype_df.transpose()
genotype_df = genotype_df.reindex(age_sex_df.index)

/tmp/ipykernel_954722/4250337195.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  genotype_df = genotype_df.replace({'0/0': 0, '1/0': 1, '0/1': 1, '1/1': 2})


In [26]:
import pandas as pd
import statsmodels.api as sm

snp = 'chr5:96783162:C:T'

genotype_series = genotype_df[[f'{snp}']]
combined_df = genotype_series.join(age_sex_df)
combined_df = combined_df.dropna()
# combined_df['intercept'] = 1.0

# 指定模型的自变量和因变量
X = combined_df.drop('group', axis=1) 
y = combined_df['group'] 

model = sm.Logit(y, X)
result = model.fit()

print(result.summary())


Optimization terminated successfully.
         Current function value: 0.463255
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  group   No. Observations:                  111
Model:                          Logit   Df Residuals:                      108
Method:                           MLE   Df Model:                            2
Date:                Thu, 19 Dec 2024   Pseudo R-squ.:                  0.2854
Time:                        14:39:13   Log-Likelihood:                -51.421
converged:                       True   LL-Null:                       -71.959
Covariance Type:            nonrobust   LLR p-value:                 1.204e-09
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
chr5:96783162:C:T     1.1691      0.941      1.242      0.214      -0.676       3.014
sex     